# Proyecto integrador I
---
Presentado por:
* Karla Orozco
* Jonathan zapata
* Juan Fernando Gallego

---
### Importar librerías

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:

import nltk

In [17]:
nltk.download('punkt')
# nltk.download('words')
# nltk.download('wordnet')
# from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /Users/jonny/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/jonny/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


---
### Cargar datos

In [3]:
company = pd.read_csv('raw/Company.csv')
company_tweet = pd.read_csv('raw/Company_Tweet.csv')
tweet = pd.read_csv('raw/Tweet.csv')

---
### Observar datos

In [4]:
company.head()

,ticker_symbol,company_name
0,AAPL,apple
1,GOOG,Google Inc
2,GOOGL,Google Inc
3,AMZN,Amazon.com
4,TSLA,Tesla Inc


In [4]:
company.shape

(6, 2)

In [5]:
company_tweet.head()

,tweet_id,ticker_symbol
0,550803612197457920,AAPL
1,550803610825928706,AAPL
2,550803225113157632,AAPL
3,550802957370159104,AAPL
4,550802855129382912,AAPL


In [6]:
company_tweet.shape

(4336445, 2)

In [5]:
tweet.head(2)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0


In [8]:
tweet.shape

(3717964, 7)

Ajustar formato fecha en la tabla "tweet":

In [6]:
tweet['post_date'] = pd.to_datetime(tweet['post_date'],unit='s')
tweet.head(2)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0


Crear una columna con la fecha hasta la unidad de días:

In [7]:
tweet['date'] = tweet['post_date'].dt.date
tweet.head(2)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,date
0,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,2015-01-01
1,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0,2015-01-01


Valores nulos en la tabla "tweet":

In [8]:
tweet.isnull().sum()

tweet_id           0
writer         47273
post_date          0
body               0
comment_num        0
retweet_num        0
like_num           0
date               0
dtype: int64

Evaluar duplicados de la columna "tweet_id" de la tabla "tweet":

In [12]:
tweet.duplicated(subset=['tweet_id']).sum()

0

In [13]:
tweet['tweet_id'].duplicated().sum()

0

---
### Traer en la tabla "tweet" las compañias de las que habla cada tweet por la columna "tweet_id":

Valores nulos en la tabla "company_tweet":

In [14]:
company_tweet.isnull().sum()

tweet_id         0
ticker_symbol    0
dtype: int64

Evaluar duplicados de la columna "tweet_id" de la tabla "company_tweet":

In [15]:
company_tweet.duplicated(subset=['tweet_id']).sum()

618481

##### Examinar duplicados:

In [9]:
company_tweet.loc[company_tweet.duplicated(subset=['tweet_id'])].head(1)

,tweet_id,ticker_symbol
1425013,550447850857828352,GOOG


In [17]:
company_tweet.loc[company_tweet['tweet_id'] == 550447850857828352]

,tweet_id,ticker_symbol
283,550447850857828352,AAPL
1425013,550447850857828352,GOOG
2145152,550447850857828352,AMZN
3239585,550447850857828352,TSLA


In [18]:
tweet.loc[tweet['tweet_id'] == 550447850857828352, 'body'].values

array(['Top 10 searched #stocks of #2014 $AAPL $FB $BABA $TSLA $BAC $GOOG $INTC $GE $TWTR $AMZN http://stks.co/g1V5S http://stks.co/h1VAb'],
      dtype=object)

Los tweets pueden hacer referencia a más de una empresa.

#### Agrupar por "tweet_id" el "ticker_symbol" de las empresas relacionadas:

In [10]:
ticker_symbol_group = company_tweet.groupby('tweet_id')['ticker_symbol'].unique()

Renombrar y mostrar la serie:

In [11]:
ticker_symbol_group.rename("ticker_symbol_group", inplace=True)
ticker_symbol_group.head(2)

tweet_id
550441509175443456    [AAPL]
550441672312512512    [AAPL]
Name: ticker_symbol_group, dtype: object

Realizar combinación mencionada entre la tabla agrupada "ticker_symbol_group" y "tweet" por la columna "tweet_id":

In [12]:
tweet = tweet.merge(ticker_symbol_group, how='left',left_on='tweet_id',right_on='tweet_id')

Revisar el tweet examinado en los duplicados de la tabla "company_tweet":

In [ ]:
tweet.loc[tweet['tweet_id'] == 550447850857828352]

In [13]:
tweet_head = tweet.head(100).copy()

In [16]:
tweet_head['body_tokenized'] = tweet_head.apply(lambda row: nltk.word_tokenize(str(row['body'])), axis=1)
tweet_head['body_tokenized'].head()

0    [lx21, made, $, 10,008, on, $, AAPL, -Check, i...
1    [Insanity, of, today, weirdo, massive, selling...
2    [S, &, P100, #, Stocks, Performance, $, HD, $,...
3    [$, GM, $, TSLA, :, Volkswagen, Pushes, 2014, ...
4    [Swing, Trading, :, Up, To, 8.91, %, Return, I...
Name: body_tokenized, dtype: object

In [ ]:
# stopwords en nltk
stop_words_nltk = set(stopwords.words('english'))
print(f'Stopwords length: {len(stop_words_nltk)}')
print(f'Stopwords: {stop_words_nltk}')

In [30]:
'compandX10'.isalpha()

False

In [31]:
'compandX'.isalpha()

True

In [11]:
# ELIMINAR tokens de long = 1
# ELIMINAR caracteres que no sean alfanumericos
# NUEVAMENTE ELIMINAR tokens de long = 1
# REMOVER tokens conformados solo por numeros, ya que no necesitamos buscar expresiones solo numéricas
# REMOVER stop words
refined_tokens_by_file = []
for (idx, tokens) in enumerate(tweet_head['body_tokenized']):
    tokens = [re.sub(r'[^A-Za-z0-9]+','',w) for w in tokens]
    # tokens=[word for word in tokens if word.isalpha()] si en vez de re.sub(r'[^A-Za-z0-9]+','',w) hace esto, que pasa?
    tokens = [w.lower() for w in tokens if len(w)>1]
    tokens = [w for w in tokens if not w.isnumeric()]
#     for i, w in reversed(list(enumerate(tokens))):
#         try:
#           # Preguntamos si el token es solo numerico y si si, lo eliminamos
#         if (w.isnumeric()):
#             tokens.pop(i)
#         except:
#             pass
    tokens = [w for w in tokens if w not in stop_words_nltk]
    refined_tokens_by_file.append(tokens)
  
  #fdist = nltk.FreqDist(refined_tokens_by_file)
  # extract top 20 words
  #topwords = fdist.most_common(20)
  #print(f"numero de palabras finales en {df['filename'][idx]} = {len(fdist)}")
  #print(f"Top 20: {topwords}")
  # graficar los 20 términos más frecuentes:
  #x,y = zip(*topwords)

  #plt.figure(figsize=(15,10))
  #plt.bar(x,y)
  #plt.xlabel("Word")
  #plt.ylabel("frecuency")
  #plt.xticks(rotation=90)
  #plt.show()
  #print('------------------------------------------')

tweet_head['body_tokenized'] = refined_tokens_by_file
tweet_head.head()


,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,body_tokenized
0,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,"[lx21, made, aapl, check, http, profitly1mnd8s..."
1,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0,"[insanity, today, weirdo, massive, selling, aa..."
2,550441732014223360,DozenStocks,2015-01-01 00:01:50,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0,"[p100, stocks, performance, hd, low, sbux, tgt..."
3,550442977802207232,ShowDreamCar,2015-01-01 00:06:47,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1,"[gm, tsla, volkswagen, pushes, record, recall,..."
4,550443807834402816,i_Know_First,2015-01-01 00:10:05,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,"[swing, trading, return, days, http, owlygdks0..."


In [12]:
# Lemmatization con NLTK

from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

lemmatized_tokens_by_file = []
for (idx, tokens) in enumerate(refined_tokens_by_file):
  lemmatized_tokens = [wordnet_lemmatizer.lemmatize(w) for w in tokens ]
  lemmatized_tokens_by_file.append(lemmatized_tokens)

  #fdist = nltk.FreqDist(lemmatized_tokens)
  # extract top 20 words
  #topwords = fdist.most_common(20)
  #print(f"numero de palabras finales en {tweet_head['body_tokenized'][idx]} = {len(fdist)}")
  #print(f"Top 20: {topwords}")
  #x,y = zip(*topwords)
  #plt.figure(figsize=(15,10))
  #plt.bar(x,y)
  #plt.xlabel("Word")
  #plt.ylabel("frecuency")
  #plt.xticks(rotation=90)
  #plt.show()
  #print('------------------------------------------')

tweet_head['lemmatized_tokens'] = lemmatized_tokens_by_file
tweet_head.head()

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,body_tokenized,lemmatized_tokens
0,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,"[lx21, made, aapl, check, http, profitly1mnd8s...","[lx21, made, aapl, check, http, profitly1mnd8s..."
1,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0,"[insanity, today, weirdo, massive, selling, aa...","[insanity, today, weirdo, massive, selling, aa..."
2,550441732014223360,DozenStocks,2015-01-01 00:01:50,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0,"[p100, stocks, performance, hd, low, sbux, tgt...","[p100, stock, performance, hd, low, sbux, tgt,..."
3,550442977802207232,ShowDreamCar,2015-01-01 00:06:47,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1,"[gm, tsla, volkswagen, pushes, record, recall,...","[gm, tsla, volkswagen, push, record, recall, t..."
4,550443807834402816,i_Know_First,2015-01-01 00:10:05,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,"[swing, trading, return, days, http, owlygdks0...","[swing, trading, return, day, http, owlygdks0,..."


In [43]:
tweet_head['list_lemma'] = tweet_head['lemmatized_tokens'].apply(
    lambda c: list(w for w in c if not any(x.isdigit() for x in w)))

In [44]:
tweet_head['list_lemma']

0     [made, aapl, check, http, learn, howtotrade, h...
1     [insanity, today, weirdo, massive, selling, aa...
2     [stock, performance, hd, low, sbux, tgt, dvn, ...
3     [gm, tsla, volkswagen, push, record, recall, t...
4     [swing, trading, return, day, http, swingtradi...
                            ...                        
95      [tsla, scty, got, ta, wait, patient, suck, huh]
96    [rt, cnbc, earlier, month, mysterious, glitch,...
97    [option, greek, critical, option, trading, htt...
98    [betting, big, cytr, gild, baba, aapl, clvs, c...
99    [goog, max, pain, maturity, price, http, maxim...
Name: list_lemma, Length: 100, dtype: object

In [46]:
tweet_head['clean_tweet'] = tweet_head['list_lemma'].apply(lambda c: ' '.join(c))

In [47]:
tweet_head['clean_tweet']

0     made aapl check http learn howtotrade http exe...
1     insanity today weirdo massive selling aapl bid...
2     stock performance hd low sbux tgt dvn ibm amzn...
3     gm tsla volkswagen push record recall tally hi...
4     swing trading return day http swingtrading for...
                            ...                        
95               tsla scty got ta wait patient suck huh
96    rt cnbc earlier month mysterious glitch caused...
97    option greek critical option trading http bitl...
98    betting big cytr gild baba aapl clvs cldx blue...
99    goog max pain maturity price http maximumpainc...
Name: clean_tweet, Length: 100, dtype: object

In [51]:
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/karlaorozco/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [52]:
tweet_head['score_vader'] = tweet_head['clean_tweet'].apply(
    lambda c: SentimentIntensityAnalyzer().polarity_scores(c)['compound'])

In [55]:
tweet_head['score_vader']

0     0.0000
1    -0.8271
2    -0.2732
3     0.0000
4     0.0000
       ...  
95   -0.4404
96   -0.2732
97   -0.3182
98    0.0000
99   -0.5106
Name: score_vader, Length: 100, dtype: float64

In [56]:
conditions = [
    (tweet_head["score_vader"] >= .05),
    (tweet_head["score_vader"] > -.05) & (tweet_head["score_vader"] < .05),
    (tweet_head["score_vader"] <= -.05),
]

choices = ['positive', 'neutral', 'negative']

tweet_head['sentiment'] = np.select(conditions, choices)

In [57]:
tweet_head['sentiment'].value_counts()

neutral     36
positive    35
negative    29
Name: sentiment, dtype: int64

In [60]:
tweet_head.head()

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,body_tokenized,lemmatized_tokens,list_lemma,clean_tweet,score_vader,sentiment
0,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,"[lx21, made, aapl, check, http, profitly1mnd8s...","[lx21, made, aapl, check, http, profitly1mnd8s...","[made, aapl, check, http, learn, howtotrade, h...",made aapl check http learn howtotrade http exe...,0.0000,neutral
1,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0,"[insanity, today, weirdo, massive, selling, aa...","[insanity, today, weirdo, massive, selling, aa...","[insanity, today, weirdo, massive, selling, aa...",insanity today weirdo massive selling aapl bid...,-0.8271,negative
2,550441732014223360,DozenStocks,2015-01-01 00:01:50,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0,"[p100, stocks, performance, hd, low, sbux, tgt...","[p100, stock, performance, hd, low, sbux, tgt,...","[stock, performance, hd, low, sbux, tgt, dvn, ...",stock performance hd low sbux tgt dvn ibm amzn...,-0.2732,negative
3,550442977802207232,ShowDreamCar,2015-01-01 00:06:47,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1,"[gm, tsla, volkswagen, pushes, record, recall,...","[gm, tsla, volkswagen, push, record, recall, t...","[gm, tsla, volkswagen, push, record, recall, t...",gm tsla volkswagen push record recall tally hi...,0.0000,neutral
4,550443807834402816,i_Know_First,2015-01-01 00:10:05,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,"[swing, trading, return, days, http, owlygdks0...","[swing, trading, return, day, http, owlygdks0,...","[swing, trading, return, day, http, swingtradi...",swing trading return day http swingtrading for...,0.0000,neutral


In [6]:
company['ticker_symbol'].values

array(['AAPL', 'GOOG', 'GOOGL', 'AMZN', 'TSLA', 'MSFT'], dtype=object)

In [8]:
import yfinance as yf

In [143]:
prices=yf.download(list(company['ticker_symbol'].values), start="2014-12-31", end="2020-12-31")['Adj Close']

[*********************100%***********************]  6 of 6 completed


In [144]:
prices.head()

,AAPL,AMZN,GOOG,GOOGL,MSFT,TSLA
Date,,,,,,
2014-12-31,24.983650,310.350006,524.958740,530.659973,40.920731,44.481998
2015-01-02,24.745998,308.519989,523.373108,529.549988,41.193836,43.862000
2015-01-05,24.048861,302.190002,512.463013,519.460022,40.815033,42.018002
2015-01-06,24.051128,295.290009,500.585632,506.640015,40.215965,42.256001
2015-01-07,24.388371,298.420013,499.727997,505.149994,40.726933,42.189999


#### Rellenar las fechas diarias dentro del rango de análisis que no aparecen en los precios descargados, con los datos no nulos de periodos previos:

Crear un DataFrame con las fechas diarias del rango de análisis como índice, nombrado "Date":

In [145]:
price=pd.DataFrame(index=pd.date_range(start="2014-12-31",end="2020-12-31"))
price.index.name='Date'

In [146]:
price.head()

""
Date
2014-12-31
2015-01-01
2015-01-02
2015-01-03
2015-01-04


Concatenar el DataFrame creado con los precios descargador por índice:

In [147]:
price=pd.concat([price,prices],axis=1)

In [148]:
price.head()

,AAPL,AMZN,GOOG,GOOGL,MSFT,TSLA
Date,,,,,,
2014-12-31,24.983650,310.350006,524.958740,530.659973,40.920731,44.481998
2015-01-01,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,24.745998,308.519989,523.373108,529.549988,41.193836,43.862000
2015-01-03,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-04,NaN,NaN,NaN,NaN,NaN,NaN


Rellenar los datos "NaN" con el los datos no nulos de la fila anterior:

In [149]:
price=price.fillna(method='ffill')

In [150]:
price.head(2)

,AAPL,AMZN,GOOG,GOOGL,MSFT,TSLA
Date,,,,,,
2014-12-31,24.98365,310.350006,524.95874,530.659973,40.920731,44.481998
2015-01-01,24.98365,310.350006,524.95874,530.659973,40.920731,44.481998


In [151]:
price.head()

,AAPL,AMZN,GOOG,GOOGL,MSFT,TSLA
Date,,,,,,
2014-12-31,24.983650,310.350006,524.958740,530.659973,40.920731,44.481998
2015-01-01,24.983650,310.350006,524.958740,530.659973,40.920731,44.481998
2015-01-02,24.745998,308.519989,523.373108,529.549988,41.193836,43.862000
2015-01-03,24.745998,308.519989,523.373108,529.549988,41.193836,43.862000
2015-01-04,24.745998,308.519989,523.373108,529.549988,41.193836,43.862000


Validar no vácios:

In [152]:
price.isnull().sum().sum()==0

True

Validar que todas las fechas díarias del rango de análisis se encuentren:

In [177]:
all(pd.date_range(start="2015-01-01",end="2020-12-31").isin(price.index))

True

Calcular el cambio porcentual en el precio por día:

In [170]:
change_price=price.pct_change()

In [171]:
change_price.head()

,AAPL,AMZN,GOOG,GOOGL,MSFT,TSLA
Date,,,,,,
2014-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2015-01-02,-0.009512,-0.005897,-0.00302,-0.002092,0.006674,-0.013938
2015-01-03,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2015-01-04,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


Restar al índice de fechas un día, correspondiente al periodo donde se evaluarán los tweets que pudieron haber tenido relación con la variación ocurrida en el día siguiente:

In [172]:
import datetime

In [173]:
change_price.index-=datetime.timedelta(days=1)

In [174]:
change_price.head()

,AAPL,AMZN,GOOG,GOOGL,MSFT,TSLA
Date,,,,,,
2014-12-30,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-31,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2015-01-01,-0.009512,-0.005897,-0.00302,-0.002092,0.006674,-0.013938
2015-01-02,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2015-01-03,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


Eliminar periodos fuera del rango de análisis:

In [175]:
change_price.drop(['2014-12-30','2014-12-31'], inplace=True)

In [178]:
change_price.head()

,AAPL,AMZN,GOOG,GOOGL,MSFT,TSLA
Date,,,,,,
2015-01-01,-0.009512,-0.005897,-0.003020,-0.002092,0.006674,-0.013938
2015-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-01-04,-0.028172,-0.020517,-0.020846,-0.019054,-0.009196,-0.042041
2015-01-05,0.000094,-0.022833,-0.023177,-0.024679,-0.014678,0.005664


#### Umbral

---
#### Exportar datos a la zona trusted

In [23]:
tweet.to_csv('trusted/data.csv',index=False)